In [ ]:
!pip install -q git+https://github.com/huggingface/transformers.git
!pip install -q tensorflow==2.1

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

torch_device = "cuda" if torch.cuda.is_available() else "cpu"
tokenizer = AutoTokenizer.from_pretrained("gpt2")
model = AutoModelForCausalLM.from_pretrained("gpt2", pad_token_id=tokenizer.eos_token_id).to(torch_device)


In [ ]:
import pandas as pd
df = pd.read_csv('/content/patent_data_A61B.csv')

In [ ]:
df['generated_abstract'] = pd.NA
def generate_abstract(claim_text):
    try:
        model_inputs = tokenizer(f'this is a patent claim:{claim_text}. Generate the abstract for this claim',return_tensors='pt').to(torch_device)
        sample_output = model.generate(
                        **model_inputs,
                        do_sample=True,
                        max_length=500,
                        top_k=40
                    )
        decoded_output = tokenizer.decode(sample_output[0], skip_special_tokens=True)
        input_prompt = tokenizer.decode(model_inputs['input_ids'][0], skip_special_tokens=True)
        cleaned_output = decoded_output[len(input_prompt):] if decoded_output.startswith(input_prompt) else decoded_output


        return cleaned_output
    except Exception as e:
        print("An error occurred:", e)
        return None

df.iloc[0:0].to_csv('GPT2_A61B.csv', index=False)

for index, row in df.iterrows():
    print(index)
    generated_abstract = generate_abstract(row['claim_text'])
    df.at[index, 'generated_abstract'] = generated_abstract if isinstance(generated_abstract, str) else 'Error or empty'
    df.iloc[[df.index.get_loc(index)]].to_csv('GPT2_A61B.csv', mode='a', header=False, index=False)
